In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "yjbds/ml-models:0.4.8"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'yjbds/ml-models:0.4.8'


In [3]:
import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx


def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["data-source", "ml"]
fn.metadata.labels = {"author": "yjb", "framework": "sklearn"}
fn.export("function.yaml")

[mlrun] 2020-05-25 01:41:34,460 function spec saved to path: function.yaml


In [7]:
from mlutils import get_vol_mount
fn.apply(get_vol_mount())

## tests

In [8]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "wine"}}

### run remotely

In [12]:
run = fn.run(NewTask(**task_params), artifact_path=mlconf.artifact_path + "/toy-data")

[mlrun] 2020-05-25 01:42:31,380 starting run tasks load toy dataset uid=b318dc5db75d414bb916fa69e175281b  -> http://mlrun-api:8080
[mlrun] 2020-05-25 01:42:31,516 Job is running in the background, pod: tasks-load-toy-dataset-vrrtw
[mlrun] 2020-05-25 01:42:34,396 warning!, server (kubeflow) and client (default-tenant) namespace dont match
[mlrun] 2020-05-25 01:42:34,402 starting local run: main.py # load_dataset
[mlrun] 2020-05-25 01:42:34,406 warning!, server (kubeflow) and client (default-tenant) namespace dont match
[mlrun] 2020-05-25 01:42:34,694 log artifact wine at /home/jovyan/data/toy-data/wine.parquet, size: 18454, db: Y

[mlrun] 2020-05-25 01:42:34,714 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...75281b,0,May 25 01:42:34,completed,tasks load toy dataset,kind=jobowner=jovyanhost=tasks-load-toy-dataset-vrrtw,,dataset=wine,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run b318dc5db75d414bb916fa69e175281b  , !mlrun logs b318dc5db75d414bb916fa69e175281b 
[mlrun] 2020-05-25 01:42:38,049 run executed, status=completed


### or locally

In [10]:
from mlrun import run_local

In [13]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset}, artifact_path=mlconf.artifact_path + "/toy-data")

[mlrun] 2020-05-25 01:42:51,963 starting run mlrun-0d5a4b-load_dataset uid=28b08f53dce947a49956e5a921f27149  -> http://mlrun-api:8080
[mlrun] 2020-05-25 01:42:52,043 log artifact wine at /home/jovyan/data/toy-data/wine.parquet, size: 18454, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...f27149,0,May 25 01:42:51,completed,mlrun-0d5a4b-load_dataset,kind=handlerowner=jovyanhost=jupyter-notebook-6447b7f894-m4vj4,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 28b08f53dce947a49956e5a921f27149 --project default , !mlrun logs 28b08f53dce947a49956e5a921f27149 --project default
[mlrun] 2020-05-25 01:42:52,078 run executed, status=completed
[mlrun] 2020-05-25 01:42:52,083 starting run mlrun-315780-load_dataset uid=4b3bdf1673ca4bbaada0b7bdc3e59e7f  -> http://mlrun-api:8080
[mlrun] 2020-05-25 01:42:52,127 log artifact iris at /home/jovyan/data/toy-data/iris.parquet, size: 5209, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...e59e7f,0,May 25 01:42:52,completed,mlrun-315780-load_dataset,kind=handlerowner=jovyanhost=jupyter-notebook-6447b7f894-m4vj4,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 4b3bdf1673ca4bbaada0b7bdc3e59e7f --project default , !mlrun logs 4b3bdf1673ca4bbaada0b7bdc3e59e7f --project default
[mlrun] 2020-05-25 01:42:52,155 run executed, status=completed
[mlrun] 2020-05-25 01:42:52,160 starting run mlrun-24e16d-load_dataset uid=44dca54f485b447199f0f68f90063c6b  -> http://mlrun-api:8080
[mlrun] 2020-05-25 01:42:52,312 log artifact breast_cancer at /home/jovyan/data/toy-data/breast_cancer.parquet, size: 147746, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...063c6b,0,May 25 01:42:52,completed,mlrun-24e16d-load_dataset,kind=handlerowner=jovyanhost=jupyter-notebook-6447b7f894-m4vj4,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run 44dca54f485b447199f0f68f90063c6b --project default , !mlrun logs 44dca54f485b447199f0f68f90063c6b --project default
[mlrun] 2020-05-25 01:42:52,365 run executed, status=completed
